In [1]:
from eoh import eoh
from eoh.utils.tyroParas import TyroParas
import tyro

programmatic_defaults = TyroParas(
    method="eoh",
    problem="ssscsp",
    llm_api_endpoint="openrouter.ai",
    llm_api_key="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",  # Use the value from the environment variable
    llm_model="google/gemini-2.0-flash-001",
    # llm_model="google/gemini-2.5-flash-preview-05-20",
    # llm_model="google/gemini-2.0-flash-lite-001",
    ec_pop_size=5,  # number of samples in each population
    ec_n_pop=5,  # number of populations
    exp_n_proc=20,  # multi-core parallel
    exp_debug_mode=False,
    # Any other parameters you want to set as a default baseline
)
params = programmatic_defaults

from eoh.problems import problems
from eoh.methods import methods

evolution = eoh.EVOL(params)

problemGenerator = problems.Probs(evolution.paras)

problem = problemGenerator.get_problem()

methodGenerator = methods.Methods(evolution.paras, problem)

method = methodGenerator.get_method()

from eoh.methods.eoh.eoh_interface_EC import InterfaceEC

interface_prob = method.prob

# interface for ec operators
interface_ec = InterfaceEC(
    method.pop_size,
    method.m,
    method.api_endpoint,
    method.api_key,
    method.llm_model,
    method.use_local_llm,
    method.llm_local_url,
    method.debug_mode,
    interface_prob,
    select=method.select,
    n_p=method.exp_n_proc,
    timeout=method.timeout,
    use_numba=method.use_numba,
)


import re
from func_timeout import func_timeout, FunctionTimedOut
from eoh.invoker import AlgorithmInvoker




DEFAULT_CLASS_NAME_HIERARCHY = ["Algorithm", "AlgorithmFIX1", "AlgorithmFIX2", "AlgorithmFIX3"]

def timed_code_evaluation(
    interface_eval, invoker: AlgorithmInvoker, timeout
):
    try:
        fitness_val, error_msg_eval = func_timeout(
            timeout,
            interface_eval.evaluate,
            args=(
                invoker,
            ),
        )
    except FunctionTimedOut:
        fitness_val = None
        error_msg_eval = f"Evaluation timed out for class '{invoker.class_to_instantiate_name}'."
    except Exception as e:
        fitness_val = None
        error_msg_eval = f"An unexpected error occurred during timed evaluation: {type(e).__name__}: {str(e)}"
    return fitness_val, error_msg_eval

def extract_python_code_from_llm_response(markdown_string):
    if not isinstance(markdown_string, str):
        return None
    match = re.search(r"```python\n(.*?)\n```", markdown_string, re.DOTALL)
    if match:
        return match.group(1).strip()
    if "```" not in markdown_string and (
        markdown_string.strip().startswith("class ") or
        markdown_string.strip().startswith("def ") or
        markdown_string.strip().startswith("import ")
        ):
        return markdown_string.strip()
    return None

def process_code_submission_with_llm_fixing(
    initial_prompt,
    initial_code,
    base_class_code_str,
    interface_eval,
    call_llm_api,
    timeout
):
    llm_conversation_history = [
        {"role": "user", "content": initial_prompt},
        {"role": "assistant", "content": initial_code},
    ]
    successful_parent_code_definitions = []
    final_successful_code_block = None
    class_name_hierarchy = DEFAULT_CLASS_NAME_HIERARCHY 
    max_attempts = len(class_name_hierarchy)
    code_for_current_attempt = initial_code
    final_fitness_achieved = None
    for attempt_idx in range(max_attempts):
        current_class_to_instantiate = class_name_hierarchy[attempt_idx]
        definitions_for_eval = successful_parent_code_definitions + [code_for_current_attempt]

        fitness_val, error_msg_eval = timed_code_evaluation(interface_eval,AlgorithmInvoker(base_class_code_str,definitions_for_eval,current_class_to_instantiate,interface_eval.prompts.prompt_func_name), timeout)
        attempt_info = {"class_name": current_class_to_instantiate, "fitness": fitness_val}
        if fitness_val is None:
            attempt_info["error"] = error_msg_eval

        if fitness_val is not None:
            print(f"SUCCESS: Evaluation successful for {current_class_to_instantiate}! Fitness: {fitness_val}")
            final_successful_code_block = "\n".join(definitions_for_eval)
            final_fitness_achieved = fitness_val
            break
        else:
            if attempt_idx == max_attempts - 1:
                print("Max attempts reached. Could not fix the code for this submission.")
                break

            next_llm_class_name = class_name_hierarchy[attempt_idx + 1]
            parent_class_for_llm_fix = current_class_to_instantiate

            fix_prompt = f"""The Python code for class `{parent_class_for_llm_fix}` has an issue.
    When this class was used in the packing algorithm, it resulted in the following error:
    {error_msg_eval}

    Instructions:
    1. Analyze the error message and the provided code for `{parent_class_for_llm_fix}`.
    2. Create a new Python class named `{next_llm_class_name}`.
    3. This new class `{next_llm_class_name}` MUST inherit from class `{parent_class_for_llm_fix}`.
    4. In the new class (`{next_llm_class_name}`), override only the specific method(s) from `{parent_class_for_llm_fix}` that are causing the error or are directly related to fixing it. Do NOT rewrite the entire `{parent_class_for_llm_fix}` class, only provide the overridden methods or necessary additions in `{next_llm_class_name}`.
    5. The primary objective is to resolve the specified error so the code can run without this error.
    6. Ensure your fixed code is robust and adheres to the original problem's requirements if the error points to a deviation.
    7. Your response should ONLY be the Python code for the new `{next_llm_class_name}` class. Do not include any explanatory text, markdown formatting for the code block itself, or anything other than the class definition.
    """
            current_fix_request_message = {"role": "user", "content": fix_prompt}
            messages_to_send_to_llm = llm_conversation_history + [current_fix_request_message]

            llm_generated_code_fix_raw = call_llm_api(messages_to_send_to_llm)
            llm_generated_code_fix = extract_python_code_from_llm_response(llm_generated_code_fix_raw)

            if llm_generated_code_fix:
                llm_conversation_history.append(current_fix_request_message)
                llm_conversation_history.append({"role": "assistant", "content": llm_generated_code_fix_raw})
                successful_parent_code_definitions.append(code_for_current_attempt)
                code_for_current_attempt = llm_generated_code_fix
            else:
                print("LLM did not return valid code. Stopping iterative fixing for this submission.")
                break
    
    status_message = "SUCCESS" if final_successful_code_block else "FAILED"
    print(f"\n--- Processing of code submission Finished: {status_message} ---")
    return current_class_to_instantiate, final_successful_code_block, final_fitness_achieved



pop = []
operator = 'i1'
select = interface_ec.select
m = interface_ec.m
evol = interface_ec.evol
interface_eval = interface_ec.interface_eval
use_numba = interface_ec.use_numba
debug = interface_ec.debug
timeout = interface_ec.timeout

p = None
final_offspring = {
    'algorithm': None, 'code': None, 'objective': None, 'other_inf': None
}

# Inlined _standalone_get_alg function
offspring_data = {
    'algorithm': None,
    'code': None,
    'prompt': None
}
p = None


prompt_content = evol.get_prompt_i1()

--- Applying post-initialization logic ---
Successfully processed 20 instances.
--- Post-initialization complete ---
----------------------------------------- 
---              Start EoH            ---
-----------------------------------------
- output folder created -
-  parameters loaded -
- Prob local loaded 
- EoH parameters loaded -
- check LLM API
remote llm api is used ...


In [2]:
print(prompt_content)

You are to solve the **Single-Stock-Size Cutting Stock Problem (SSSCSP)**. You are given a list of item **types** (each with dimensions and a **quantity**) and the dimensions of a single standard container. Your goal is to pack all items into the minimum number of these identical containers.
Design a novel algorithm that, at each step, selects an available item type, **chooses one of 6 possible orientations**, and finds a valid position for it in a container.
        **Primary Objective: Minimize the total number of stock containers used.**

        Constraints:
        1. **Single Container Type:** All containers are identical. No weight or support constraints apply.
        2. **Complete Placement:** All items of all types must be packed.
        3. **Item Orientation:** Items can be rotated. There are 6 possible orientations.
        4. **No Overlap:** Items cannot overlap.
        5. **Boundaries:** Items must be placed fully inside the container.
        6. **Immutable Inputs:** Y

In [2]:
#dump to json
import json
# load responses from json
with open('code_2_0_flash.json', 'r') as f:
    all_code = json.load(f)

In [3]:
from eoh.methods.eoh.eoh_interface_EC import process_code_submission_with_llm_fixing
for i in range(len(all_code)):
    print(f"Code {i+1}/{len(all_code)}:")
    code = all_code[i]
    process_code_submission_with_llm_fixing(
            initial_prompt=prompt_content,
            initial_code=code,
            base_class_code_str=interface_eval.base_class_code,
            interface_eval=interface_eval,
            call_llm_api=evol.interface_llm.interface_llm.call_llm_api,
            timeout=timeout
        )

Code 1/20:
Attempt 1: Evaluation timed out for class 'Algorithm'.
Attempt 2: Evaluation timed out for class 'AlgorithmFIX1'.
SUCCESS: Evaluation successful for AlgorithmFIX2! Fitness: 120.9

--- Processing of code submission Finished: SUCCESS ---
Code 2/20:
Attempt 1: Evaluation timed out for class 'Algorithm'.
Attempt 2: Evaluation timed out for class 'AlgorithmFIX1'.
SUCCESS: Evaluation successful for AlgorithmFIX2! Fitness: 120.9

--- Processing of code submission Finished: SUCCESS ---
Code 3/20:
Attempt 1: Evaluation timed out for class 'Algorithm'.
Attempt 2: Evaluation timed out for class 'AlgorithmFIX1'.
SUCCESS: Evaluation successful for AlgorithmFIX2! Fitness: 32.7

--- Processing of code submission Finished: SUCCESS ---
Code 4/20:
SUCCESS: Evaluation successful for Algorithm! Fitness: 27.2

--- Processing of code submission Finished: SUCCESS ---
Code 5/20:
Attempt 1: Evaluation timed out for class 'Algorithm'.
Attempt 2: Evaluation timed out for class 'AlgorithmFIX1'.
SUCCESS

In [27]:
initial_code = """
import math

class Algorithm:


    def __init__(self):
        # State cache to avoid re-calculating spaces if the input hasn't changed.
        self._state_cache = {}
        self.epsilon = 1e-5

    def place_item(self, unplaced_items, trucks_in_use, truck_type):
        best_placement = None
        best_placement_score = (float('inf'),) 

        prioritized_items = self._get_prioritized_items(unplaced_items)

        for truck_index, truck in enumerate(trucks_in_use):
            empty_spaces = self._get_empty_spaces_for_truck(truck_index, truck, truck_type)
            if not empty_spaces:
                continue

            truck_best_placement, truck_best_score = self._find_best_block_in_spaces(
                empty_spaces, prioritized_items, unplaced_items, truck_type
            )

            if truck_best_placement and truck_best_score < best_placement_score:
                best_placement_score = truck_best_score
                best_placement = (truck_index, *truck_best_placement)

        if best_placement:
            return best_placement

        if not trucks_in_use or best_placement is None:
            initial_space = {'pos': (0, 0, 0), 'dims': truck_type}
            
            new_truck_placement, _ = self._find_best_block_in_spaces(
                [initial_space], prioritized_items, unplaced_items, truck_type
            )

            if new_truck_placement:
                return (-1, *new_truck_placement)

        raise ValueError("Could not place any item, even in a new truck.")

    def _get_prioritized_items(self, unplaced_items):
        ranked_items = []
        for i, item in enumerate(unplaced_items):
            if item.get('quantity', 0) > 0:
                dims = sorted([item['length'], item['width'], item['height']])
                score = (-dims[0], -item['quantity'], -dims[2])
                ranked_items.append({'score': score, 'item_index': i})

        ranked_items.sort(key=lambda x: x['score'])
        return ranked_items

    def _find_best_block_in_spaces(self, spaces, prioritized_items, unplaced_items, truck_type):
        best_placement_for_truck = None
        best_score_for_truck = (float('inf'),)

        spaces.sort(key=lambda s: (s['pos'][2], s['pos'][1], s['pos'][0]))

        for space in spaces:
            space_pos = space['pos']
            space_dims = space['dims']
            space_vol = space_dims[0] * space_dims[1] * space_dims[2]

            for ranked_item in prioritized_items:
                item_idx = ranked_item['item_index']
                item_type = unplaced_items[item_idx]
                item_priority_score = ranked_item['score']

                for orientation, item_dims in self._get_orientations(item_type):
                    
                    # <<< FIX: ADDED SAFETY CHECK >>>
                    # First, ensure the item placed at the space's origin is fully
                    # within the truck's absolute boundaries. This prevents errors from
                    # edge cases in the space splitting logic.
                    if (space_pos[0] + item_dims[0] > truck_type[0] + self.epsilon or
                        space_pos[1] + item_dims[1] > truck_type[1] + self.epsilon or
                        space_pos[2] + item_dims[2] > truck_type[2] + self.epsilon):
                        continue

                    # Second, check if at least one item can fit in the space itself
                    if (item_dims[0] <= space_dims[0] + self.epsilon and
                        item_dims[1] <= space_dims[1] + self.epsilon and
                        item_dims[2] <= space_dims[2] + self.epsilon):
                        
                        num_h = math.floor(space_dims[2] / item_dims[2])
                        num_w = math.floor(space_dims[1] / item_dims[1])
                        
                        if num_h < 1 or num_w < 1:
                            continue

                        num_in_block = num_h * num_w
                        block_volume = num_in_block * (item_dims[0] * item_dims[1] * item_dims[2])
                        volume_utilization = block_volume / space_vol if space_vol > 0 else 0

                        current_score = (-volume_utilization, item_priority_score, space_pos)

                        if current_score < best_score_for_truck:
                            best_score_for_truck = current_score
                            best_placement_for_truck = (
                                item_idx,
                                space_pos[0],
                                space_pos[1],
                                space_pos[2],
                                orientation
                            )
        
        return best_placement_for_truck, best_score_for_truck

    def _get_orientations(self, item_type):
        L, W, H = item_type['length'], item_type['width'], item_type['height']
        return [
            (0, (L, W, H)), (1, (L, H, W)),
            (2, (W, L, H)), (3, (W, H, L)),
            (4, (H, L, W)), (5, (H, W, L))
        ]

    def _get_empty_spaces_for_truck(self, truck_index, truck, truck_type):
        occupied_items = truck.get('occupied_volumes', [])
        state_key = (truck_index, tuple(
            (it['x'], it['y'], it['z'], it['length'], it['width'], it['height'])
            for it in sorted(occupied_items, key=lambda i: (i['x'], i['y'], i['z']))
        ))

        if state_key in self._state_cache:
            return self._state_cache[state_key]

        initial_space = {'pos': (0, 0, 0), 'dims': truck_type}
        empty_spaces = [initial_space]

        for item in occupied_items:
            item_pos = (item['x'], item['y'], item['z'])
            item_dims = (item['length'], item['width'], item['height'])
            
            newly_generated_spaces = []
            for space in empty_spaces:
                newly_generated_spaces.extend(self._split_space(space, (item_pos, item_dims)))
            empty_spaces = newly_generated_spaces

        self._state_cache[state_key] = empty_spaces
        return empty_spaces

    def _split_space(self, space, blocker):
        s_pos, s_dims = space['pos'], space['dims']
        b_pos, b_dims = blocker[0], blocker[1]

        intersect_min_x = max(s_pos[0], b_pos[0])
        intersect_min_y = max(s_pos[1], b_pos[1])
        intersect_min_z = max(s_pos[2], b_pos[2])

        intersect_max_x = min(s_pos[0] + s_dims[0], b_pos[0] + b_dims[0])
        intersect_max_y = min(s_pos[1] + s_dims[1], b_pos[1] + b_dims[1])
        intersect_max_z = min(s_pos[2] + s_dims[2], b_pos[2] + b_dims[2])

        if (intersect_max_x <= intersect_min_x + self.epsilon or
            intersect_max_y <= intersect_min_y + self.epsilon or
            intersect_max_z <= intersect_min_z + self.epsilon):
            return [space]

        new_spaces = []
        
        if s_pos[0] < b_pos[0] - self.epsilon:
            new_spaces.append({'pos': (s_pos[0], s_pos[1], s_pos[2]), 
                               'dims': (b_pos[0] - s_pos[0], s_dims[1], s_dims[2])})
        if (s_pos[0] + s_dims[0]) > (b_pos[0] + b_dims[0]) + self.epsilon:
            new_x = b_pos[0] + b_dims[0]
            new_spaces.append({'pos': (new_x, s_pos[1], s_pos[2]),
                               'dims': ((s_pos[0] + s_dims[0]) - new_x, s_dims[1], s_dims[2])})
        if s_pos[1] < b_pos[1] - self.epsilon:
            new_spaces.append({'pos': (intersect_min_x, s_pos[1], s_pos[2]),
                               'dims': (intersect_max_x - intersect_min_x, b_pos[1] - s_pos[1], s_dims[2])})
        if (s_pos[1] + s_dims[1]) > (b_pos[1] + b_dims[1]) + self.epsilon:
            new_y = b_pos[1] + b_dims[1]
            new_spaces.append({'pos': (intersect_min_x, new_y, s_pos[2]),
                               'dims': (intersect_max_x - intersect_min_x, (s_pos[1] + s_dims[1]) - new_y, s_dims[2])})
        if s_pos[2] < b_pos[2] - self.epsilon:
            new_spaces.append({'pos': (intersect_min_x, intersect_min_y, s_pos[2]),
                               'dims': (intersect_max_x - intersect_min_x, intersect_max_y - intersect_min_y, b_pos[2] - s_pos[2])})
        if (s_pos[2] + s_dims[2]) > (b_pos[2] + b_dims[2]) + self.epsilon:
            new_z = b_pos[2] + b_dims[2]
            new_spaces.append({'pos': (intersect_min_x, intersect_min_y, new_z),
                               'dims': (intersect_max_x - intersect_min_x, intersect_max_y - intersect_min_y, (s_pos[2] + s_dims[2]) - new_z)})

        return [s for s in new_spaces if all(d > self.epsilon for d in s['dims'])]
"""

In [28]:
code_for_current_attempt = initial_code
DEFAULT_CLASS_NAME_HIERARCHY = ["Algorithm", "AlgorithmFIX1", "AlgorithmFIX2", "AlgorithmFIX3"]
class_name_hierarchy = DEFAULT_CLASS_NAME_HIERARCHY 
current_class_to_instantiate = class_name_hierarchy[0]
definitions_for_eval = [code_for_current_attempt]
interface_eval.evaluate(AlgorithmInvoker(interface_eval.base_class_code,definitions_for_eval,current_class_to_instantiate, interface_eval.prompts.prompt_func_name))



(17.21276595744681, 'Evaluation successful')

In [19]:
from SSSCSP import PackingCONST, GetData


getData = GetData(47)
interface_eval.instance_data = getData.generate_instances()

Successfully processed 47 instances.


In [29]:
17.21276595744681*47

809.0